## Which conference is the best at winning tight games?



### How can we find out which conference is the best at winning tight games?

### We need to look at play-by-play data to figure this out, because if we just look at final scores we won't truly know whether a game was close. 

## Step 1
### Let's start by looking at [a game we know was close](https://www.youtube.com/watch?v=SdFB3OGUaGU) to get a sense of the play-by-play data.

##### (Sorry for the reminder, Longhorn fans!)

In [0]:
from pandas.io import gbq
project_id = '[YOUR_PROJECT_ID]'


In [12]:
n_iowa_vs_texas_q = """
SELECT
  away_market,
  home_market,
  away_pts,
  home_pts,
  elapsed_time_sec,
  period,
  game_clock,
  team_market,
  jersey_num,
  event_type,
  points_scored
FROM
  `bigquery-public-data.ncaa_basketball.mbb_pbp_ncaa`
WHERE
  game_id = "703-504-2016-03-18"
  AND (home_pts != 0
    AND away_pts != 0)
ORDER BY
  elapsed_time_sec DESC
"""

n_iowa_vs_texas = gbq.read_gbq(query=n_iowa_vs_texas_q, dialect ='standard', project_id=project_id)
n_iowa_vs_texas.head(25)



Requesting query... ok.
Job ID: job_AGIm7wr7VTvoUcHXxY9FeIsz6dlq
Query running...
Query done.
Cache hit.

Retrieving results...
Got 81 rows.

Total time taken 0.56 s.
Finished at 2018-03-16 08:20:17.


,away_market,home_market,away_pts,home_pts,elapsed_time_sec,period,game_clock,team_market,jersey_num,event_type,points_scored
0,Northern Iowa,Texas,75,72,2400,2,00:00,Northern Iowa,04,GOOD,3
1,Northern Iowa,Texas,72,72,2397,2,00:03,Texas,01,GOOD,2
2,Northern Iowa,Texas,72,70,2389,2,00:11,Northern Iowa,11,GOOD,1
3,Northern Iowa,Texas,71,70,2374,2,00:26,Northern Iowa,02,GOOD,2
4,Northern Iowa,Texas,69,70,2355,2,00:45,Texas,44,GOOD,1
5,Northern Iowa,Texas,69,69,2264,2,02:16,Texas,21,GOOD,2
6,Northern Iowa,Texas,69,67,2225,2,02:55,Northern Iowa,05,GOOD,1
7,Northern Iowa,Texas,68,67,2197,2,03:23,Northern Iowa,20,GOOD,1
8,Northern Iowa,Texas,67,67,2197,2,03:23,Northern Iowa,20,GOOD,1
9,Northern Iowa,Texas,66,67,2182,2,03:38,Texas,21,GOOD,1


### What a shot!

## Step 2

### Down to business. Let's decide that a "close game" is any game where the score difference is:

### (1) Less than 4 at any point in the last 5 minutes OR
### (2) Less than 7 for >2 of the last 5 minutes OR
### (3) Less than 10 for >3 of the last 5 minutes

##### Note that the NCAA started collecting play-by-play in the middle of the 2009 season. Due to discrepancies in when teams started collecting, it's cleaner to start with the 2010 season ("AND season > 2009").

### Here is how we can find how many games there are like this:

In [13]:
close_games_q = """
  #late_plays shows all scoring plays, and also entries for the end of regulation time and the five-minutes remaining mark
WITH
  late_plays AS (
  SELECT
    season,
    game_id,
    home_division_alias,
    away_division_alias,
    away_pts,
    home_pts,
    elapsed_time_sec
  FROM
    `bigquery-public-data.ncaa_basketball.mbb_pbp_ncaa`
  UNION ALL
  SELECT
    season,
    game_id,
    home_division_alias,
    away_division_alias,
    MAX(away_pts),
    MAX(home_pts),
    2100 AS elapsed_time_sec
  FROM
    `bigquery-public-data.ncaa_basketball.mbb_pbp_ncaa`
  WHERE
    elapsed_time_sec <= 2100
  GROUP BY
    season,
    game_id,
    home_division_alias,
    away_division_alias
  UNION ALL
  SELECT
    season,
    game_id,
    home_division_alias,
    away_division_alias,
    MAX(away_pts),
    MAX(home_pts),
    2400 AS elapsed_time_sec
  FROM
    `bigquery-public-data.ncaa_basketball.mbb_pbp_ncaa`
  WHERE
    elapsed_time_sec <= 2400
  GROUP BY
    season,
    game_id,
    home_division_alias,
    away_division_alias ),
  diffs AS (
  SELECT
    game_id,
    away_pts,
    home_pts,
    ABS(away_pts - home_pts) AS diff,
    elapsed_time_sec,
    MAX(elapsed_time_sec) OVER (PARTITION BY game_id ORDER BY elapsed_time_sec DESC, away_pts + home_pts DESC ROWS BETWEEN 1 PRECEDING AND CURRENT ROW) AS next_update
  FROM
    late_plays
  WHERE
    home_division_alias = "D1"
    AND away_division_alias = "D1"
    AND season > 2009
    AND (home_pts != 0
      AND away_pts != 0)
    AND elapsed_time_sec >= 2100
    AND elapsed_time_sec <= 2400
  ORDER BY
    elapsed_time_sec DESC,
    away_pts + home_pts DESC )
SELECT
  game_id,
  a AS one_poss_game_time_sec,
  b AS two_poss_game_time_sec,
  c AS three_poss_game_time_sec
FROM (
  SELECT
    game_id,
    SUM(IF(diff < 4,
        time_before_change,
        0)) AS a,
    SUM(IF(diff < 7,
        time_before_change,
        0)) AS b,
    SUM(IF(diff < 10,
        time_before_change,
        0)) AS c,
    IF((SUM(IF(diff < 4,
            time_before_change,
            0)) > 0
        OR SUM(IF(diff < 7,
            time_before_change,
            0)) > 120
        OR SUM(IF(diff < 10,
            time_before_change,
            0)) > 180),
      TRUE,
      FALSE) AS close
  FROM (
    SELECT
      game_id,
      away_pts,
      home_pts,
      elapsed_time_sec,
      ABS(away_pts - home_pts) AS diff,
      next_update - elapsed_time_sec AS time_before_change
    FROM
      diffs )
  GROUP BY
    game_id )
WHERE
  close = TRUE
"""

close_games = gbq.read_gbq(query=close_games_q, dialect ='standard', project_id=project_id)
close_games.head(25)

Requesting query... ok.
Job ID: job_ODf3Q6wR7LEvutQQ9Oi4Zdz4jDex
Query running...
Query done.
Cache hit.

Retrieving results...
Got 22221 rows.

Total time taken 3.1 s.
Finished at 2018-03-16 08:20:21.


,game_id,one_poss_game_time_sec,two_poss_game_time_sec,three_poss_game_time_sec
0,365-2711-2015-12-02,0,0,213
1,331-47-2017-01-03,0,0,236
2,678-228-2016-11-26,0,0,196
3,490-83-2015-12-05,0,0,209
4,310-116-2013-01-13,0,0,233
5,128-404-2016-01-26,0,0,257
6,156-77-2011-01-22,0,0,207
7,1014-253-2010-12-04,0,0,232
8,311-310-2016-03-17,0,0,182
9,460-180-2017-03-04,0,0,209


## Step 3
### Let's do a couple sanity checks. How many games are there total?

In [14]:
total_games_q = """
SELECT
  DISTINCT game_id
FROM
  `bigquery-public-data.ncaa_basketball.mbb_pbp_ncaa`
WHERE
  home_division_alias = "D1"
  AND away_division_alias = "D1"
  AND season > 2009
"""

total_games = gbq.read_gbq(query=total_games_q, dialect ='standard', project_id=project_id)
total_games.head(25)

Requesting query... ok.
Job ID: job_f5d_OKWvIpmYBhBU0miIjwNLz20L
Query running...
Query done.
Processed: 852.6 MB
Standard price: $0.00 USD

Retrieving results...
Got 43560 rows.

Total time taken 3.22 s.
Finished at 2018-03-16 08:20:25.


,game_id
0,172-363-2015-12-22
1,172-315-2013-12-14
2,172-738-2014-01-04
3,172-503-2014-12-19
4,172-813-2013-02-02
5,172-368-2013-11-16
6,172-554-2011-03-04
7,172-738-2018-01-10
8,172-167-2013-03-08
9,172-469-2012-11-13


## Step 4

### So what percentage of games are close?

In [15]:
print "close games: " + str(len(close_games))
print "all games: " + str(len(total_games))
print "% close = " + str(float(len(close_games))/float(len(total_games)))

close games: 22221
all games: 43560
% close = 0.510123966942


### That seems reasonable!

## Step 5

### Now we need to add up how many of these close games each conference won and lost.

In [16]:
close_wins_by_conf_q = """
  #late_plays shows all scoring plays, and also entries for the end of regulation time and the five-minutes remaining mark
WITH
  late_plays AS (
  SELECT
    season,
    game_id,
    home_division_alias,
    away_division_alias,
    away_pts,
    home_pts,
    elapsed_time_sec
  FROM
    `bigquery-public-data.ncaa_basketball.mbb_pbp_ncaa`
  UNION ALL
  SELECT
    season,
    game_id,
    home_division_alias,
    away_division_alias,
    MAX(away_pts),
    MAX(home_pts),
    2100 AS elapsed_time_sec
  FROM
    `bigquery-public-data.ncaa_basketball.mbb_pbp_ncaa`
  WHERE
    elapsed_time_sec <= 2100
  GROUP BY
    season,
    game_id,
    home_division_alias,
    away_division_alias
  UNION ALL
  SELECT
    season,
    game_id,
    home_division_alias,
    away_division_alias,
    MAX(away_pts),
    MAX(home_pts),
    2400 AS elapsed_time_sec
  FROM
    `bigquery-public-data.ncaa_basketball.mbb_pbp_ncaa`
  WHERE
    elapsed_time_sec <= 2400
  GROUP BY
    season,
    game_id,
    home_division_alias,
    away_division_alias ),
  diffs AS (
  SELECT
    game_id,
    away_pts,
    home_pts,
    ABS(away_pts - home_pts) AS diff,
    elapsed_time_sec,
    MAX(elapsed_time_sec) OVER (PARTITION BY game_id ORDER BY elapsed_time_sec DESC, away_pts + home_pts DESC ROWS BETWEEN 1 PRECEDING AND CURRENT ROW) AS next_update
  FROM
    late_plays
  WHERE
    home_division_alias = "D1"
    AND away_division_alias = "D1"
    AND season > 2009
    AND (home_pts != 0
      AND away_pts != 0)
    AND elapsed_time_sec >= 2100
    AND elapsed_time_sec <= 2400
  ORDER BY
    elapsed_time_sec DESC,
    away_pts + home_pts DESC ),
  close_games AS (
  SELECT
    game_id
  FROM (
    SELECT
      game_id,
      SUM(IF(diff < 4,
          time_before_change,
          0)) AS a,
      SUM(IF(diff < 7,
          time_before_change,
          0)) AS b,
      SUM(IF(diff < 10,
          time_before_change,
          0)) AS c,
      IF((SUM(IF(diff < 4,
              time_before_change,
              0)) > 0
          OR SUM(IF(diff < 7,
              time_before_change,
              0)) > 120
          OR SUM(IF(diff < 10,
              time_before_change,
              0)) > 180),
        TRUE,
        FALSE) AS close
    FROM (
      SELECT
        game_id,
        away_pts,
        home_pts,
        elapsed_time_sec,
        ABS(away_pts - home_pts) AS diff,
        next_update - elapsed_time_sec AS time_before_change
      FROM
        diffs )
    GROUP BY
      game_id )
  WHERE
    close = TRUE ),
  close_games_scores AS (
  SELECT
    scores.scheduled_date,
    scores.game_id,
    scores.away_conf_alias,
    scores.home_conf_alias,
    MAX(scores.away_pts) AS final_away_pts,
    MAX(scores.home_pts) AS final_home_pts
  FROM
    `bigquery-public-data.ncaa_basketball.mbb_pbp_ncaa` AS scores
  INNER JOIN
    close_games
  ON
    close_games.game_id = scores.game_id
  GROUP BY
    scheduled_date,
    game_id,
    away_conf_alias,
    home_conf_alias )
SELECT
  conf,
  MAX(IF(home_or_away='home'
      AND won,
      cnt,
      NULL)) h_won,
  MAX(IF(home_or_away='home'
      AND NOT won,
      cnt,
      NULL)) h_lost,
  MAX(IF(home_or_away='away'
      AND won,
      cnt,
      NULL)) a_won,
  MAX(IF(home_or_away='away'
      AND NOT won,
      cnt,
      NULL)) a_lost,
  SUM(IF(NOT won,
      cnt,
      NULL)) loss,
  SUM(IF(won,
      cnt,
      NULL)) won,
  SUM(cnt) total,
  SUM(IF(won,
      cnt,
      NULL))/SUM(cnt) AS pct
FROM (
  SELECT
    home_conf_alias AS conf,
    'home' AS home_or_away,
    final_home_pts > final_away_pts AS won,
    COUNT(*) cnt
  FROM
    close_games_scores
  GROUP BY
    conf,
    home_or_away,
    won
  UNION ALL
  SELECT
    away_conf_alias AS conf,
    'away' AS home_or_away,
    final_away_pts > final_home_pts AS won,
    COUNT(*) cnt
  FROM
    close_games_scores
  GROUP BY
    conf,
    home_or_away,
    won )
GROUP BY
  conf
ORDER BY
  pct DESC
"""

close_wins_by_conf = gbq.read_gbq(query=close_wins_by_conf_q, dialect ='standard', project_id=project_id)
print close_wins_by_conf

Requesting query... ok.
Job ID: job_QafLW1hxTqww3gZq3Sb_lBofvtT4
Query running...
Query done.
Cache hit.

Retrieving results...
Got 32 rows.

Total time taken 0.53 s.
Finished at 2018-03-16 08:20:27.
         conf  h_won  h_lost  a_won  a_lost  loss   won  total       pct
0         AAC    511     333    340     370   703   851   1554  0.547619
1     BIGEAST    403     282    295     299   581   698   1279  0.545739
2         ACC    655     420    396     475   895  1051   1946  0.540082
3       PAC12    557     306    283     410   716   840   1556  0.539846
4       BIG12    432     266    249     326   592   681   1273  0.534957
5       BIG10    591     396    367     452   848   958   1806  0.530454
6         MWC    447     279    301     390   669   748   1417  0.527876
7         SEC    627     380    335     504   884   962   1846  0.521127
8         A10    570     402    399     499   901   969   1870  0.518182
9      SUMMIT    269     158    209     300   458   478    936  0.5106

### That looks OK. But there's still something wrong: we're including all those games between two teams in the same conference. These games will always give that conference a win and a loss. If the conference wins a lot (more than 50%) of their close games against other conferences, all of their close games within the conference will pull their winning percentage down towards 50%. 

## Step 7
### So let's take those games out.

In [17]:
close_wins_by_conf2_q = """
  #late_plays shows all scoring plays, and also entries for the end of regulation time and the five-minutes remaining mark
WITH
  late_plays AS (
  SELECT
    season,
    game_id,
    home_division_alias,
    away_division_alias,
    away_pts,
    home_pts,
    elapsed_time_sec
  FROM
    `bigquery-public-data.ncaa_basketball.mbb_pbp_ncaa`
  UNION ALL
  SELECT
    season,
    game_id,
    home_division_alias,
    away_division_alias,
    MAX(away_pts),
    MAX(home_pts),
    2100 AS elapsed_time_sec
  FROM
    `bigquery-public-data.ncaa_basketball.mbb_pbp_ncaa`
  WHERE
    elapsed_time_sec <= 2100
  GROUP BY
    season,
    game_id,
    home_division_alias,
    away_division_alias
  UNION ALL
  SELECT
    season,
    game_id,
    home_division_alias,
    away_division_alias,
    MAX(away_pts),
    MAX(home_pts),
    2400 AS elapsed_time_sec
  FROM
    `bigquery-public-data.ncaa_basketball.mbb_pbp_ncaa`
  WHERE
    elapsed_time_sec <= 2400
  GROUP BY
    season,
    game_id,
    home_division_alias,
    away_division_alias ),
  diffs AS (
  SELECT
    game_id,
    away_pts,
    home_pts,
    ABS(away_pts - home_pts) AS diff,
    elapsed_time_sec,
    MAX(elapsed_time_sec) OVER (PARTITION BY game_id ORDER BY elapsed_time_sec DESC, away_pts + home_pts DESC ROWS BETWEEN 1 PRECEDING AND CURRENT ROW) AS next_update
  FROM
    late_plays
  WHERE
    home_division_alias = "D1"
    AND away_division_alias = "D1"
    AND season > 2009
    AND (home_pts != 0
      AND away_pts != 0)
    AND elapsed_time_sec >= 2100
    AND elapsed_time_sec <= 2400
  ORDER BY
    elapsed_time_sec DESC,
    away_pts + home_pts DESC ),
  close_games AS (
  SELECT
    game_id
  FROM (
    SELECT
      game_id,
      SUM(IF(diff < 4,
          time_before_change,
          0)) AS a,
      SUM(IF(diff < 7,
          time_before_change,
          0)) AS b,
      SUM(IF(diff < 10,
          time_before_change,
          0)) AS c,
      IF((SUM(IF(diff < 4,
              time_before_change,
              0)) > 0
          OR SUM(IF(diff < 7,
              time_before_change,
              0)) > 120
          OR SUM(IF(diff < 10,
              time_before_change,
              0)) > 180),
        TRUE,
        FALSE) AS close
    FROM (
      SELECT
        game_id,
        away_pts,
        home_pts,
        elapsed_time_sec,
        ABS(away_pts - home_pts) AS diff,
        next_update - elapsed_time_sec AS time_before_change
      FROM
        diffs )
    GROUP BY
      game_id )
  WHERE
    close = TRUE ),
  close_games_scores AS (
  SELECT
    scores.scheduled_date,
    scores.game_id,
    scores.away_conf_alias,
    scores.home_conf_alias,
    MAX(scores.away_pts) AS final_away_pts,
    MAX(scores.home_pts) AS final_home_pts
  FROM
    `bigquery-public-data.ncaa_basketball.mbb_pbp_ncaa` AS scores
  INNER JOIN
    close_games
  ON
    close_games.game_id = scores.game_id
  WHERE
    home_conf_alias != away_conf_alias # <<<<<<<<<<<<<<<<<<<< take out for non-conference analysis
  GROUP BY
    scheduled_date,
    game_id,
    away_conf_alias,
    home_conf_alias )
SELECT
  conf,
  MAX(IF(home_or_away='home'
      AND won,
      cnt,
      NULL)) h_won,
  MAX(IF(home_or_away='home'
      AND NOT won,
      cnt,
      NULL)) h_lost,
  MAX(IF(home_or_away='away'
      AND won,
      cnt,
      NULL)) a_won,
  MAX(IF(home_or_away='away'
      AND NOT won,
      cnt,
      NULL)) a_lost,
  SUM(IF(NOT won,
      cnt,
      NULL)) loss,
  SUM(IF(won,
      cnt,
      NULL)) won,
  SUM(cnt) total,
  SUM(IF(won,
      cnt,
      NULL))/SUM(cnt) AS pct
FROM (
  SELECT
    home_conf_alias AS conf,
    'home' AS home_or_away,
    final_home_pts > final_away_pts AS won,
    COUNT(*) cnt
  FROM
    close_games_scores
  GROUP BY
    conf,
    home_or_away,
    won
  UNION ALL
  SELECT
    away_conf_alias AS conf,
    'away' AS home_or_away,
    final_away_pts > final_home_pts AS won,
    COUNT(*) cnt
  FROM
    close_games_scores
  GROUP BY
    conf,
    home_or_away,
    won )
GROUP BY
  conf
ORDER BY
  pct DESC
"""

close_wins_by_conf2 = gbq.read_gbq(query=close_wins_by_conf2_q, dialect ='standard', project_id=project_id)
print close_wins_by_conf2

Requesting query... ok.
Job ID: job_LyMY2yLjl_HbJCSpvCDJMNMvOblB
Query running...
Query done.
Cache hit.

Retrieving results...
Got 32 rows.

Total time taken 0.59 s.
Finished at 2018-03-16 08:20:28.
         conf  h_won  h_lost  a_won  a_lost  loss  won  total       pct
0       PAC12    251     116     93     104   220  344    564  0.609929
1         ACC    322     166    142     142   308  464    772  0.601036
2       BIG12    202     109     92      96   205  294    499  0.589178
3     BIGEAST    245     134    147     141   275  392    667  0.587706
4         AAC    330     181    188     189   370  518    888  0.583333
5       BIG10    281     149    120     142   291  401    692  0.579480
6         MWC    204     115    137     147   262  341    603  0.565506
7         SEC    272     154    109     149   303  381    684  0.557018
8         A10    285     191    188     214   405  473    878  0.538724
9         MVC    203     142    127     163   305  330    635  0.519685
10     S

## Once we remove intra-conference games, the result is clear: the PAC-12 is the best at winning close games (against other conferences)!